In [1]:
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import matplotlib.axes as axe
import pandas as pd
import datetime as dt
import gurobipy as gp
from gurobipy import GRB
import cvxpy as cp

import random
from itertools import chain, combinations, tee
import time

plt.rcParams['text.usetex'] = True

# Functions

In [2]:
def demand_name_by_group_index(index):
    list_demand_names = ["Demand (eligible group, 1)", "Demand (eligible group, 2)", \
                         "Demand (ineligible group, 1)", "Demand (ineligible group, 2)", \
                         "Demand (ineligible group, 3)"]
    return list_demand_names[index]

def VoT_name_by_group_index(index):
    list_demand_names = ["VoT (eligible group, 1)", "VoT (eligible group, 2)", \
                         "VoT (ineligible group, 1)", "VoT (ineligible group, 2)", \
                         "VoT (ineligible group, 3)"]
    return list_demand_names[index]


# Groups, Routes to Edges:

In [3]:
directory_path = '../data/data_income_percentage_VoT___101_N_Sep_to_Nov_2024/'
df_data = pd.read_csv(directory_path + 'data_cities_od_VoTs_demands_1.csv')

# df_od_flow_data
# df_data

In [4]:
dict_data = {}

for column_name_full in list(df_data.columns):
    if column_name_full == "Data Category":
        categories_list = df_data[column_name_full].tolist()
    else:
        dict_data[int(column_name_full)] = {}
        for category_index, category in enumerate(categories_list):
            if category == "Start City Index" or category == "End City Index":
                dict_data[int(column_name_full)][category] \
                    = int(df_data[column_name_full].tolist()[category_index])
            elif category == "Start City" or category == "End City":
                dict_data[int(column_name_full)][category] \
                    = df_data[column_name_full].tolist()[category_index]
            else:
#                 print("category:", category)
                dict_data[int(column_name_full)][category] \
                    = float(df_data[column_name_full].tolist()[category_index])


In [5]:
dict_data

{0: {'Start City Index': 0,
  'End City Index': 0,
  'Start City': 'Palo Alto',
  'End City': 'Palo Alto',
  'O-D Flow (Max Entropy)': 612.539616936298,
  'Demand (eligible group, 1)': 47.77809012103125,
  'VoT (eligible group, 1)': 0.03197423570019724,
  'Demand (eligible group, 2)': 27.564282762133406,
  'VoT (eligible group, 2)': 0.10238603988603988,
  'Demand (ineligible group, 1)': 115.15744798402402,
  'VoT (ineligible group, 1)': 0.2754407051282051,
  'Demand (ineligible group, 2)': 134.75871572598555,
  'VoT (ineligible group, 2)': 0.5809294871794872,
  'Demand (ineligible group, 3)': 287.28108034312373,
  'VoT (ineligible group, 3)': 1.8596449415012848},
 1: {'Start City Index': 0,
  'End City Index': 1,
  'Start City': 'Palo Alto',
  'End City': 'East Palo Alto',
  'O-D Flow (Max Entropy)': 98.03491986336807,
  'Demand (eligible group, 1)': 7.64672374934271,
  'VoT (eligible group, 1)': 0.03197423570019724,
  'Demand (eligible group, 2)': 4.411571393851563,
  'VoT (eligible g

In [6]:
cities_dict = {}
for od_info in list(dict_data.values()):
    if od_info["Start City Index"] not in list(cities_dict.keys()):
        cities_dict[od_info["Start City Index"]] = od_info["Start City"]
    if od_info["End City Index"] not in list(cities_dict.keys()):
        cities_dict[od_info["End City Index"]] = od_info["End City"]

cities_list = list(cities_dict.values())

# cities_dict

In [7]:
od_to_edges_array = np.zeros((len(list(dict_data.keys())), 2))

for od_index, od_info in dict_data.items():
    od_to_edges_array[od_index, 0] = int(cities_list.index(od_info["Start City"]))
    od_to_edges_array[od_index, 1] = int(cities_list.index(od_info["End City"]))

# od_to_edges_array

In [8]:
num_groups_per_od = 5

demand_array = np.zeros((len(list(dict_data.keys())), num_groups_per_od))
VoT_array_base = np.zeros((len(list(dict_data.keys())), num_groups_per_od))

for od_index, od_value in dict_data.items():
    for group_index in range(num_groups_per_od):
        demand_name = demand_name_by_group_index(group_index)
        VoT_name = VoT_name_by_group_index(group_index)
        
        demand_array[od_index, group_index] = od_value[demand_name]
        VoT_array_base[od_index, group_index] = od_value[VoT_name]

# print(demand_array)
# VoT_array_base

In [9]:
directory_path = '../data/data_income_percentage_VoT___101_N_Sep_to_Nov_2024/'

VoT_array = np.zeros((VoT_array_base.shape[0], VoT_array_base.shape[1], 5))

T = 5
for t in range(T):
    df_perturbation_data = pd.read_csv(directory_path + 'perturbations_1_' + str(t) + '.csv')
    perturbation_array = df_perturbation_data.to_numpy()[:, 1:]
    VoT_array[:, :, t] = VoT_array_base * perturbation_array
    
# VoT_array_base
# perturbation_array

# Download Latency Parameters Data

In [10]:
directory_path_latency = '../data/pems_latency_inference___101_N_Sep_to_Nov_2024/'
df_latency_params = pd.read_csv(directory_path_latency + 'latency_params.csv')

# list(df_latency_params.loc[:, "Palo Alto"])

In [11]:
dict_latency_params = {}

city_list = list(df_latency_params.columns)[1:]

for city in city_list:
#     if city != "Belmont":
    if 1 == 1:
        dict_latency_params[city] = {}
        dict_latency_params[city]["Flow (at bend)"] = df_latency_params.loc[:, city][0]
        dict_latency_params[city]["Latency (at bend)"] = df_latency_params.loc[:, city][1]
        dict_latency_params[city]["Slope (after bend)"] = df_latency_params.loc[:, city][2]

dict_latency_params

{'Palo Alto': {'Flow (at bend)': 861.9885,
  'Latency (at bend)': 1.326448252,
  'Slope (after bend)': 0.000782666},
 'East Palo Alto': {'Flow (at bend)': 1001.517857,
  'Latency (at bend)': 2.213126553,
  'Slope (after bend)': 0.000584484},
 'Redwood City': {'Flow (at bend)': 881.1846667,
  'Latency (at bend)': 4.892192375,
  'Slope (after bend)': 0.001563724},
 'Belmont': {'Flow (at bend)': 1278.948125,
  'Latency (at bend)': 1.199911179,
  'Slope (after bend)': 0.001994138},
 'San Mateo': {'Flow (at bend)': 1034.092826,
  'Latency (at bend)': 5.541006284,
  'Slope (after bend)': 0.002147262},
 'Burlingame': {'Flow (at bend)': 845.15,
  'Latency (at bend)': 1.503111345,
  'Slope (after bend)': 0.000306601},
 'Millbrae': {'Flow (at bend)': 853.1818182,
  'Latency (at bend)': 2.384328452,
  'Slope (after bend)': 0.000321856}}

# General CBCP Equilibrium Solver

## (Special Case) Quartic Polynomial Latency Functions

In [12]:
# grad = np.array([0, 1, 2, 3, 4])
grad = np.array([2, 4, 0, 1, 3])

for id_temp, entry_temp in enumerate(grad):
    print("id_temp, entry_temp:", id_temp, entry_temp)


id_temp, entry_temp: 0 2
id_temp, entry_temp: 1 4
id_temp, entry_temp: 2 0
id_temp, entry_temp: 3 1
id_temp, entry_temp: 4 3


In [13]:
# np.array([1, 2, 3]) * np.array([6, 5, 3])

In [49]:
# def coeff_from_input(coeff_input, num_edges, num_gp_lanes):
#     # Goal: Differentiate between express and general purpose lanes.
    
#     assert np.all(coeff_input >= 0.0), "All entries of coeff_input must be non-negative."
#     assert len(coeff_input.shape) == 2, "coeff_input should be a matrix, i.e., 2-D array"
#     assert coeff_input.shape[0] == 3, "Latency functions are assumed to be piecewise linear / affine with 3 parameters."
#     assert coeff_input.shape[1] == num_edges, "Latency functions should be defined across all edges."
    
#     coeff_length = 3
    
#     ex_to_gp_multiplier = np.array([1, 1/num_gp_lanes, num_gp_lanes]).reshape((coeff_length, 1)) \
#                             @ np.ones((1, num_edges))
    
#     coeff_full = np.zeros((coeff_length, num_edges, 2))
#     coeff_full[:, :, 0] = coeff_input
#     coeff_full[:, :, 1] = coeff_input * ex_to_gp_multiplier
    
#     return coeff_full

def latency_max(flow_max, coeff):
    
    assert np.all(coeff >= 0.0), "coeff should be non-negative"
    assert len(coeff.shape) == 1, "coeff should be a 1-D array."
    assert coeff.shape[0] == 3, "Latency functions are assumed to be piecewise linear / affine with 3 parameters."
    
    return coeff[0] + max(coeff[1] * (flow_max - coeff[2]), 0)

def welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, tau, \
                demand_array, VoT_array, num_el, od_to_edges_array, y, \
                coeff_input):

#     coeff = coeff_from_input(coeff_input, num_edges, num_gp_lanes)
    
    coeff = coeff_input
    
    assert len(od_to_edges_array.shape) == 2, "od_to_edges should be 2-dimensional."
    assert od_to_edges_array.shape[1] == 2, "od_to_edges second dimension should be for start and end edges."
    
    edge_to_od_dict = {}
    for e in range(num_edges):
        edge_to_od_dict[e] = [k for k in list(range(int(od_to_edges_array.shape[0]) )) \
                               if od_to_edges_array[k, 0] <= e <= od_to_edges_array[k, 1]]

    num_groups = demand_array.shape[1]
    num_in = num_groups - num_el
    assert num_in >= 0, "We must have num_in >= 0."
    
    el_indices = list(range(num_el))
    in_indices = list(range(num_el, num_groups))
    
#     print()
#     print("tau.shape[0]:", tau.shape[0])
#     print("num_edges:", num_edges)
#     print()

    assert len(tau.shape) == 2, "tau should be 2-dimensional."
    assert tau.shape[0] == num_edges, "toll vector's first axis length must equal the number of edges."
    assert tau.shape[1] == T, "toll vector's second axis length must equal the time horizon."
    
    ## Compute lane flows:
    
    x = np.zeros((num_edges, 2, T))
    for e in range(num_edges):
        for t in range(T):
            x[e, 0, t] += sum(y[(od, g, e, 0, t)] for od in edge_to_od_dict[e] \
                              for g in range(num_groups))
    for e in range(num_edges):
        for t in range(T):
            x[e, 1, t] += sum( (y[(od, g, e, 0, t)] + y[(od, g, e, 1, t)]) for od in edge_to_od_dict[e] \
                              for g in el_indices)
            x[e, 1, t] += sum(y[(od, g, e, 1, t)] for od in edge_to_od_dict[e] \
                              for g in in_indices)
            
    print()
    print("in_indices:", in_indices)
    print("el_indices:", el_indices)
    print()
    
    ## Compute lane latencies:
    
    ell = np.zeros((num_edges, 2, T))
    for e in range(num_edges):
        for t in range(T):
            ell[e, 0, t] = coeff[0, e] + coeff[1, e] * max(x[e, 0, t] - coeff[2, e], 0)
            ell[e, 1, t] = coeff[0, e] + coeff[1, e] * max(x[e, 1, t]/num_gp_lanes - coeff[2, e], 0)

    obj_E = sum( y[(od, g, e, 0, t)] * VoT_array[od, g, t] * ell[e, 0, t] \
                for e in range(num_edges) for od in edge_to_od_dict[e] \
                for g in el_indices for t in range(T) ) \
            + sum( y[(od, g, e, 1, t)] * (VoT_array[od, g, t] * ell[e, 0, t] + tau[e, t]) \
                for e in range(num_edges) for od in edge_to_od_dict[e] \
                for g in el_indices for t in range(T) ) \
            + sum( y[(od, g, e, 2, t)] * VoT_array[od, g, t] * ell[e, 1, t] \
                  for e in range(num_edges) for od in edge_to_od_dict[e] \
                  for g in el_indices for t in range(T) )
    obj_I = sum( y[(od, g, e, 0, t)] * (VoT_array[od, g, t] * ell[e, 0, t] + tau[e, t]) \
                for e in range(num_edges) for od in edge_to_od_dict[e] \
                for g in in_indices for t in range(T) ) \
            + sum( y[(od, g, e, 1, t)] * VoT_array[od, g, t] * ell[e, 1, t] \
                for e in range(num_edges) for od in edge_to_od_dict[e] \
                  for g in in_indices for t in range(T) )
    obj_R = sum( y[(od, g, e, 0, t)] * tau[e, t] \
                for e in range(num_edges) for od in edge_to_od_dict[e] \
                for g in in_indices for t in range(T) )

    welfare = lambda_E * obj_E - lambda_R * obj_R + lambda_I * obj_I

#     print()
#     print("obj_E:", obj_E)
#     print("obj_R:", obj_R)
#     print("obj_I:", obj_I)
#     print("welfare:", welfare)
#     print()
    
    return welfare, obj_E, obj_R, obj_I


# Latency, total, throughout the entire time horizon. 

def latency_total(T, num_edges, lambda_E, lambda_R, lambda_I, tau, \
                  demand_array, VoT_array, num_el, y, od_to_edges_array, \
                  coeff_input):
    
#     coeff = coeff_from_input(coeff_input, num_edges, num_gp_lanes)

    coeff = coeff_input
    
    assert len(od_to_edges_array.shape) == 2, "od_to_edges should be 2-dimensional."
    assert od_to_edges_array.shape[1] == 2, "od_to_edges' second dimension should be for start and end edges."
    
    edge_to_od_dict = {}
    for edge in range(num_edges):
        edge_to_od_dict[e] = [k for k in list(range(int(od_to_edges_array.shape[0]) )) \
                               if od_to_edges_array[k, 0] <= e <= od_to_edges_array[k, 1]]
    
    # In full:
    # y indices: (od, group, edge, "lane", time)

    num_groups = demand_array.shape[1]
    num_in = num_groups - num_el
    assert num_in >= 0, "We must have num_in >= 0."
    
    el_indices = list(range(num_el))
    in_indices = list(range(num_el, num_groups))
    
#     print()
#     print("tau.shape[0]:", tau.shape[0])
#     print("num_edges:", num_edges)
#     print()

    assert len(tau.shape) == 2, "tau should be 2-dimensional."
    assert tau.shape[0] == num_edges, "toll vector's first axis length must equal the number of edges."
    assert tau.shape[1] == T, "toll vector's second axis length must equal the time horizon."
    
    ## Compute lane flows:
    
    x = np.zeros((num_edges, 2, T))
    for e in range(num_edges):
        for t in range(T):
            x[e, 0, t] += sum(y[(od, g, e, 0, t)] for od in edge_to_od_dict[e] \
                              for g in range(num_groups))
    for e in range(num_edges):
        for t in range(T):
            x[e, 1, t] += sum( (y[(od, g, e, 0, t)] + y[(od, g, e, 1, t)]) for od in edge_to_od_dict[e] \
                              for g in el_indices)
            x[e, 1, t] += sum(y[(od, g, e, 1, t)] for od in edge_to_od_dict[e] \
                              for g in in_indices)
    
    ## Compute lane latencies:
    
    ell = np.zeros((num_edges, 2, T))
    for e in range(num_edges):
        for t in range(T):
            ell[e, 0, t] = coeff[0, e] + coeff[1, e] * max(x[e, 0, t] - coeff[2, e], 0)
            ell[e, 1, t] = coeff[0, e] + coeff[1, e] * max(x[e, 1, t]/num_gp_lanes - coeff[2, e], 0)

    latency = sum( x[e, k, t] * ell[e, k, t] \
                for e in range(num_edges) for k in range(2) for t in range(T) )

#     print()
#     print("obj_E:", obj_E)
#     print("obj_R:", obj_R)
#     print("obj_I:", obj_I)
#     print("welfare:", welfare)
#     print()
    
    return latency

In [39]:
# arr = np.arange(5)
# arr.reshape((5, 1))

In [50]:
def proj_tau_B(T, num_edges, tau, B, od_to_edges_list_full, tau_max, B_max = 1.0, B_min = 0.1):

#     print()
#     print("tau.shape[0]:", tau.shape[0])
#     print("num_edges:", num_edges)
#     print()

    assert tau.shape[0] == num_edges, "tau must have length equal to the number of edges."
    assert tau_max.shape[0] == num_edges, "tau_max must have length equal to the number of edges."
    
    tau_feas = cp.Variable((num_edges, T))
#     B_feas = cp.Variable(1)
    
    func = cp.sum_squares(tau_feas - tau)

    objective = cp.Minimize(func)

    constraints = []
    constraints += [tau_feas >= 0.0]
#     constraints += [B_feas >= 0.0]
    constraints += [tau_feas <= tau_max]
#     constraints += [B_feas <= B_max]

#     constraints += [B_feas <= sum(tau_feas[e, t] for e in od_to_edges_list_full[od] for t in range(T)) \
#                     for od in range(len(od_to_edges_list_full))]
    
    prob = cp.Problem(objective, constraints)
    result = prob.solve()

#     print()
#     print("tau_feas.value:", np.round(np.maximum(tau_feas.value, 0.0), decimals=4))
#     print()

    B_max_list = [sum(tau_feas.value[e, t] for e in od_to_edges_list_full[od] for t in range(T)) \
                            for od in range(len(od_to_edges_list_full))]
#     B_max_wrt_tau_feas = min([B_max for B_max in B_max_list if B_max >= B_min])
    B_max_wrt_tau_feas = max(B_max_list)
    B_feas = min(B, B_max_wrt_tau_feas)
    
    print()
    print("B:", B)
    print("B_max_wrt_tau_feas:", B_max_wrt_tau_feas)
    print("B_feas:", B_feas)
    print()

    return np.round(np.maximum(tau_feas.value, 0.0), decimals=4), \
            np.round(np.maximum(B_feas, 0.0), decimals=4)
    
#     return tau_feas.value, B_feas



In [51]:
# # OLD def proj_tau_B:

# def proj_tau_B(T, num_edges, tau, B, od_to_edges_list_full, tau_max, B_max = 1.0):

# #     print()
# #     print("tau.shape[0]:", tau.shape[0])
# #     print("num_edges:", num_edges)
# #     print()

#     assert tau.shape[0] == num_edges, "tau must have length equal to the number of edges."
#     assert tau_max.shape[0] == num_edges, "tau_max must have length equal to the number of edges."
    
#     tau_feas = cp.Variable((num_edges, T))
#     B_feas = cp.Variable(1)
    
#     func = cp.sum_squares(tau_feas - tau) + (B_feas - B)**2

#     objective = cp.Minimize(func)

#     constraints = []
#     constraints += [tau_feas >= 0.0]
#     constraints += [B_feas >= 0.0]
#     constraints += [tau_feas <= tau_max.reshape((num_edges, 1)) * np.ones((1, T))]
#     constraints += [B_feas <= B_max]

#     constraints += [B_feas <= sum(tau_feas[e, t] for e in od_to_edges_list_full[od]) \
#                     for od in range(len(od_to_edges_list_full)) for t in range(T)]
    
#     prob = cp.Problem(objective, constraints)
#     result = prob.solve()

# #     print()
# #     print("tau_feas.value:", np.round(np.maximum(tau_feas.value, 0.0), decimals=4))
# #     print()

#     return np.round(np.maximum(tau_feas.value, 0.0), decimals=4), \
#             np.round(np.maximum(B_feas.value, 0.0), decimals=4)

# Chinmay's Algorithm:

## Convex Program for CBCP and DBCP Equilibria:

In [52]:
## Steps
# 1: Define variables
# 2: Define objective
# 3: Define constraints
# 4: Define problem
# 5: Solve problem
# 6: Extract values

In [53]:
# num_edges = 8
# od_to_edges_array = np.array([[0, 0], [0, 3], [0, 7], [1, 5], [1, 7], [3, 3], [3, 6]])

# od_to_edges_list_full = [list(range(od_to_edges_array[od, 0], od_to_edges_array[od, 1] + 1)) \
#                       for od in range(od_to_edges_array.shape[0])]

# edge_to_od = [[od for od in range(od_to_edges_array.shape[0]) if e in od_to_edges_list_full[od]] \
#             for e in range(num_edges)]

# print(od_to_edges_list_full)
# print()
# print(edge_to_od)

In [54]:

# Below: For quartic latency functions:
# Latency Function: a_4 x^4 + a_3 x^3 + a_2 x^2 + a_1 x + a_0

def solve_CBCP_direct(T, num_edges, num_gp_lanes, tau, B, od_to_edges_array, \
                      demand_array, VoT_array, num_el, coeff_input):
    
#     print("tau (in solve_CBCP_direct):", tau)
    
#     coeff = coeff_from_input(coeff_input, num_edges, num_gp_lanes)

    coeff = coeff_input
    
    assert VoT_array.shape[2] == T, "We should have VoT_array.shape[2] == T."
    assert demand_array.shape == VoT_array[:, :, 0].shape, "demand_array and VoT_array[:, :, 0] should have identical shape."
    assert np.all(demand_array > 0.0), "Each entry of demand_array must be strictly positive."
    assert np.all(tau >= 0.0), "Each entry of tau must be non-negative."
    assert num_el <= demand_array.shape[1], "num_el, the number of eligible income groups, should not exceed \
                                            demand_array.shape[1], which should equal the number of income groups."
    
    num_groups = demand_array.shape[1]
    
    ## Variable indices:
    # y indices: (od, income group, edge, "lane", time)
    # y indices: (od, income group, edge, "lane", time)

    el_indices = list(range(num_el))
    in_indices = list(range(num_el, num_groups))
    group_indices = list(range(num_groups))
    
    od_to_edges_list_full = [list(range(int(od_to_edges_array[od, 0]), int(od_to_edges_array[od, 1]) + 1)) \
                          for od in range(od_to_edges_array.shape[0])]
    
    edge_to_ods = [[od for od in range(od_to_edges_array.shape[0]) if e in od_to_edges_list_full[od]] \
                   for e in range(num_edges)]
    
    num_od = len(od_to_edges_list_full)
    
    # Variables:
    y = {}
    for od in range(num_od):
        for e in od_to_edges_list_full[od]:
            for t in range(T):
                for g in el_indices:
                    for k in [0, 1, 2]:
                        y[(od, g, e, k, t)] = cp.Variable(1)
                for g in in_indices:
                    for k in [0, 1]:
                        y[(od, g, e, k, t)] = cp.Variable(1)
    
    x = {}
    for e in range(num_edges):
        for k in [0, 1]:
            for t in range(T):
                x[(e, k, t)] = cp.Variable(1)

    # Objective:
    func = 0.0
    for e in range(num_edges):
        for t in range(T):
            func += coeff[0, e] * x[e, 0, t] \
                    + 0.5 * coeff[1, e] * cp.square(cp.maximum(x[e, 0, t] - coeff[2, e], 0))
            func += coeff[0, e] * x[e, 1, t] \
                    + 0.5 * coeff[1, e] * cp.square(cp.maximum(x[e, 1, t]/num_gp_lanes - coeff[2, e], 0))
    
#     print("VoT_array.shape:", VoT_array.shape)
#     print("el_indices:", el_indices)
#     print("in_indices:", in_indices)
    
    for od in range(num_od):
        for e in od_to_edges_list_full[od]:
            for t in range(T):
                for g in el_indices:
                    func += tau[e, t] * y[(od, g, e, 1, t)] / VoT_array[od, g, t]
                for g in in_indices:
                    func += tau[e, t] * y[(od, g, e, 0, t)] / VoT_array[od, g, t]

    objective = cp.Minimize(func)
    
    # Constraints:
    constraints = []
    
    constraints += [y[(od, g, e, k, t)] >= 0.0 for od in range(num_od) \
                    for e in od_to_edges_list_full[od] for g in el_indices  \
                    for k in [0, 1, 2] for t in range(T)]
    constraints += [y[(od, g, e, k, t)] >= 0.0 for od in range(num_od) \
                    for e in od_to_edges_list_full[od] for g in in_indices  \
                    for k in [0, 1] for t in range(T)]
    constraints += [x[(e, k, t)] >= 0.0 for e in range(num_edges) \
                    for k in [0, 1] for t in range(T)]
    
#     print()
#     print("demand_array:", demand_array)
#     print()
#     print("B:", B)
#     print()
#     print("el_indices:", el_indices)
#     print()
#     print("in_indices:", in_indices)
#     print()
#     print("edge_to_ods:", edge_to_ods)
#     print()
#     print("od_to_edges_list_full:", od_to_edges_list_full)
#     print()
    
    od_g_e_t_list = []

    for e in range(num_edges):
        for t in range(T):
            
            ## Edge contraints:
            constraints += [sum( y[(od, g, e, 0, t)] + y[(od, g, e, 1, t)] for od in edge_to_ods[e] for g in el_indices) \
                                + sum( y[(od, g, e, 0, t)] for od in edge_to_ods[e] for g in in_indices ) \
                                == x[(e, 0, t)] ]
            constraints += [sum( y[(od, g, e, 2, t)] for od in edge_to_ods[e] for g in el_indices) \
                                + sum( y[(od, g, e, 1, t)] for od in edge_to_ods[e] for g in in_indices ) \
                                == x[(e, 1, t)] ]

            ## Group flow constraints:
            for od in edge_to_ods[e]:
                for g in el_indices:
#                     print("(od, g, e, t):", od, g, e, t)
                    assert (od, g, e, t) not in od_g_e_t_list, "Each (od, g, e, t) should occur only once."
                    od_g_e_t_list += [(od, g, e, t)]
        
#                     if (od, g, e, t) == (0, 0, 0, 0):
#                     if (e, t) == (2, 0) or (2, 1) or (2, 3):
#                         constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1, 2]) == demand_array[od, g]]
                    
#                 for g in in_indices:
# #                     print("(od, g, e, t):", od, g, e, t)
#                     assert (od, g, e, t) not in od_g_e_t_list, "Each (od, g, e, t) should occur only once."
#                     od_g_e_t_list += [(od, g, e, t)]
                    
#                     constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1]) == demand_array[od, g]]

            constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1, 2]) == demand_array[od, g] \
                            for od in edge_to_ods[e] for g in el_indices]
            constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1]) == demand_array[od, g] \
                            for od in edge_to_ods[e] for g in in_indices]
    
    constraints += [sum(y[(od, g, e, 0, t)] * tau[e, t] for e in od_to_edges_list_full[od] for t in range(T)) \
                    <= B * demand_array[od, g] for od in range(num_od) for g in el_indices]
    
    # Problem:
    prob = cp.Problem(objective, constraints)
    
    # Solve:
#     result = prob.solve(solver = cp.MOSEK, verbose = True)
    result = prob.solve(solver = cp.MOSEK)
    
#     for variable in prob.variables():
#         print("Variable %s" % variable.name())
    
    assert prob.status != "infeasible", "problem.status should not be infeasible."
    assert prob.status != "unbounded", "problem.status should not be unbounded."
    print()
    print("prob.status:", prob.status)

    # Extract Values:
    y_values = {}
    for e in range(num_edges):
        for od in edge_to_ods[e]:
            for t in range(T):
                for g in el_indices:
                    for k in [0, 1, 2]:
#                         print("(od, g, e, k, t):", od, g, e, k, t)
#                         print("y[(od, g, e, k, t)].value:", y[(od, g, e, k, t)].value)
                        y_values[(od, g, e, k, t)] = max(y[(od, g, e, k, t)].value[0], 0.0)
                for g in in_indices:
                    for k in [0, 1]:
#                         print("(od, g, e, k, t):", od, g, e, k, t)
#                         print("y[(od, g, e, k, t)].value:", y[(od, g, e, k, t)].value)
                        y_values[(od, g, e, k, t)] = max(y[(od, g, e, k, t)].value[0], 0.0)

    return y_values


## <font color='blue'>Implement Zeroth-order Gradient Descent:</font> 

In [55]:
## Functions defined above:

# def proj_tau_B(T, num_edges, tau, B, od_to_edges_list_full, tau_max = 1.0, B_max = 1.0):

# def solve_CBCP_direct(T, num_edges, num_gp_lanes, tau, B, od_to_edges_array, \
#                       demand_array, VoT_array, num_el, coeff_input):

#     def welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, tau, \
#                 demand_array, VoT_array, num_el, od_to_edges_array, y, \
#                 coeff_input = np.array([0.0, 0.0, 0.0, 0.0, 1.0])):

In [56]:
# coeff_input
# demand_array.shape

In [75]:
time_1 = time.time()

T = 5

num_el = 2
num_groups = demand_array.shape[1]

assert VoT_array.shape[2] == T, "We should have VoT_array.shape[2] == T"
assert demand_array.shape == VoT_array[:, :, 0].shape, "demand_array and VoT_array[:, :, 0] should have the same shape."

num_ods = demand_array.shape[0]
group_indices = list(range(demand_array.shape[1]))
# num_edges = 6
num_edges = 7
num_gp_lanes = 3

# tau = np.array([0.2, 0.6, 0.5, 0.8, 0.4, 0.5])
# assert tau.shape[0] == num_edges, "tau must have a num_edges number of entries."

# B = 0.8
a = np.array([1E-3, 1E-3, 0.0, 0.0, 0.0])

error_bound = 1E-2
diffs_num_cols = 5

# lambda_E, lambda_R, lambda_I = 1.0, 1.0, 1.0
lambda_E, lambda_R, lambda_I = 1.0, 1.0, 1.0
# lambda_E, lambda_R, lambda_I = 1.0, 1.5, 1.0

demand_edges_array = np.zeros(num_edges)

# demand_array_temp = np.ones(demand_array.shape)
# demand_array_temp = np.random.uniform(low=0.05, high=0.5, size=demand_array.shape)
demand_array_temp = demand_array


# TODO: Modify coeff_input:

## coeff_input: const, slope, x-coordinate of transition point

# coeff_input = np.array([19.4, 0.1256, 0.786*1650]).reshape((3, 1)) @ np.ones((1, num_edges))


coeff_input = np.zeros((3, num_edges))
for counter, city in enumerate(dict_latency_params.keys()):
    coeff_input[0, counter] = dict_latency_params[city]["Latency (at bend)"]
    coeff_input[1, counter] = dict_latency_params[city]["Slope (after bend)"]
    coeff_input[2, counter] = dict_latency_params[city]["Flow (at bend)"]


# def solve_CBCP_direct(T, num_edges, num_gp_lanes, tau, B, od_to_edges_array, \
#                       demand_array, VoT_array, num_el, coeff_input):
# Already defined: T, num_edges, num_gp_lanes, tau, B, od_to_edges_array, \
#                       demand_array, VoT_array

    
# print("od_to_edges_array[1, 0]:", int(od_to_edges_array[1, 0]))
# print("range(od_to_edges_array[1, 0], od_to_edges_array[1, 1] + 1):",\
#      range(int(od_to_edges_array[1, 0]), int(od_to_edges_array[1, 1]) + 1))

od_to_edges_list_full = [list(range(int(od_to_edges_array[od, 0]), int(od_to_edges_array[od, 1]) + 1)) \
                         for od in range(num_ods)]
edge_to_ods = [[od for od in range(od_to_edges_array.shape[0]) if e in od_to_edges_list_full[od]] \
               for e in range(num_edges)]
for e in range(num_edges):
    demand_edges_array[e] = sum([np.sum(demand_array_temp[od, :]) for od in range(num_ods) \
                                 if od in edge_to_ods[e]])

el_indices = list(range(num_el))
in_indices = list(range(num_el, num_groups))

tau_max_from_latency = np.zeros((num_edges, T))
VoT_max_el = np.zeros((num_edges, T))
for e in range(num_edges):
    for t in range(T):
        VoT_max_el[e, t] = max([VoT_array[od_index, group_index, t] for od_index in edge_to_ods[e] \
                                for group_index in el_indices])
        tau_max_from_latency[e, t] = VoT_max_el[e, t] * (latency_max(demand_edges_array[e], coeff_input[:, e]) - coeff_input[0, e]) \

tau_max_monetary_value = 15.0
# fraction_tau_max = tau_max_monetary_value / np.max(tau_max_from_latency)

tau_upper_limit = np.minimum(tau_max_from_latency, tau_max_monetary_value)
B_upper_limit = max([sum(tau_upper_limit[e, t] for e in od_to_edges_list_full[od] for t in range(T) ) \
                     for od in range(len(od_to_edges_list_full))])

print("tau_upper_limit:", tau_upper_limit)
print()
print("B_upper_limit:", B_upper_limit)

d = num_edges * T + 1
# num_iters = 1000
# num_iters = 100
num_iters = 200
# num_iters = 3
tau = np.zeros((num_edges, T, num_iters))
tau_perturbed = np.zeros((num_edges, T, num_iters))

B = np.zeros(num_iters)
B_perturbed = np.zeros(num_iters)

welfare_list = []

fraction_tau_max = 0.5
fraction_B_max = 0.5

# tau[:, :, 0] = fraction_tau_max * tau_upper_limit.reshape((num_edges, 1)) @ np.ones((1, T))
tau[:, :, 0] = np.random.triangular(np.zeros((num_edges, T)), tau_upper_limit * fraction_tau_max, tau_upper_limit)

# B[0] = fraction_B_max * B_upper_limit
B_upper_limit_init = max([sum(tau[e, t, 0] for e in od_to_edges_list_full[od] for t in range(T) ) \
                     for od in range(len(od_to_edges_list_full))])

B[0] = np.random.triangular(0.0, 0.5 * fraction_B_max * B_upper_limit_init, fraction_B_max * B_upper_limit_init)

print()
print("tau[:, :, 0]:", tau[:, :, 0])
print()
print("B_upper_limit_init:", B_upper_limit_init)
print()
print("B[0]:", B[0])

od_to_edges_list_full = [list(range(int(od_to_edges_array[od, 0]), int(od_to_edges_array[od, 1]) + 1 )) \
                         for od in range(od_to_edges_array.shape[0])]


# # To disable when restarting from scratch
# tau[:, :, 0] = tau_next_init
# B[0] = B_next_init
# eta[0] = eta_bar * (index_next_init+1)**(-1/2) * d**(-1)
# delta[0] = delta_bar * (index_next_init+1)**(-1/4) * d**(-1/2)

tau_upper_limit: [[0.31942364 0.31942364 0.31942364 0.31942364 0.31942364]
 [0.24983105 0.24983105 0.24983105 0.24983105 0.24983105]
 [0.99953381 0.99953381 0.99953381 0.99953381 0.99953381]
 [1.02805916 1.02805916 1.02805916 1.02805916 1.02805916]
 [1.6042613  1.6042613  1.6042613  1.6042613  1.6042613 ]
 [0.19224693 0.19224693 0.19224693 0.19224693 0.19224693]
 [0.21782897 0.21782897 0.21782897 0.21782897 0.21782897]]

B_upper_limit: 23.055924301525522

tau[:, :, 0]: [[0.09998507 0.1892127  0.28656039 0.09893891 0.04365383]
 [0.04052379 0.06717643 0.20833568 0.05774396 0.14996386]
 [0.79108695 0.66320932 0.98227094 0.06576718 0.78828095]
 [0.53572692 0.79683829 0.42663121 0.23536505 0.30208404]
 [0.74332596 0.46434049 1.20133794 1.00004619 0.38546419]
 [0.03842111 0.13033896 0.10387492 0.07271414 0.07829966]
 [0.15111417 0.1409052  0.11370494 0.09536731 0.13508253]]

B_upper_limit_init: 11.683693175470623

B[0]: 3.6500274995601307


In [76]:
# B[0] = 0.01

delta = np.zeros(num_iters)
eta = np.zeros(num_iters)
eta_bar = 2E-6
delta_bar = 0.01

for i in range(num_iters-1):
    
    print()
    print("Iter:", i)
    
    eta[i] = eta_bar * (i+1)**(-1/2) * d**(-1)
    delta[i] = delta_bar * (i+1)**(-1/4) * d**(-1/2)
    w_i_unnormalized = np.random.randn(d)
    w_i = w_i_unnormalized / np.linalg.norm(w_i_unnormalized)
#     print("w_i:", w_i)
    tau_perturbed[:, :, i] = tau[:, :, i] + delta[i] * w_i[:-1].reshape((num_edges, T))
    B_perturbed[i] = B[i] + delta[i] * w_i[-1]
    
#     if tau_perturbed[:, :, i] < B_perturbed[i] or tau_perturbed[:, :, i] < 0 or B_perturbed[i] < 0:

    print("i:\n", i)
    print("tau[:, :, i], before projection:\n", tau[:, :, i])
    print("B[i], before projection:\n", B[i])
    print("tau_perturbed[:, :, i], before projection:\n", tau_perturbed[:, :, i])
    print("B_perturbed[i], before projection:\n", B_perturbed[i])

    tau_perturbed[:, :, i], B_perturbed[i] = proj_tau_B(T, num_edges, tau_perturbed[:, :, i], B_perturbed[i], \
                                                        od_to_edges_list_full, \
                                                        tau_max = tau_upper_limit, B_max = B_upper_limit)
    
    print("tau[:, :, i], after projection:\n", tau[:, :, i])
    print("B[i], after projection:\n", B[i])
    print("tau_perturbed[:, :, i], after projection:\n", tau_perturbed[:, :, i])
    print("B_perturbed[i], after projection:\n", B_perturbed[i])

    # TODO: Remove "network":

    y_values = solve_CBCP_direct(T, num_edges, num_gp_lanes, \
                                 tau[:, :, i], B[i], od_to_edges_array, \
                                 demand_array_temp, VoT_array, num_el, coeff_input)

    y_perturbed_values = solve_CBCP_direct(T, num_edges, num_gp_lanes, \
                                           tau_perturbed[:, :, i], B_perturbed[i], od_to_edges_array, \
                                           demand_array_temp, VoT_array, num_el, coeff_input)
    
#     def welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, tau, \
#                 demand_array, VoT_array, num_el, od_to_edges_array, y, \
#                 coeff_input):

    welfare, obj_E, obj_R, obj_I = welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, \
                                               tau[:, :, i], demand_array_temp, VoT_array, num_el, od_to_edges_array, \
                                               y = y_values, coeff_input = coeff_input)
    
    welfare_perturbed, obj_E_perturbed, obj_R_perturbed, obj_I_perturbed \
        = welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, \
                      tau_perturbed[:, :, i], demand_array_temp, VoT_array, num_el, od_to_edges_array, \
                      y = y_perturbed_values, coeff_input = coeff_input)
    
    print("welfare:", welfare)
    print("obj_E:", obj_E)
    print("obj_R:", obj_R)
    print("obj_I:", obj_I)
    print()
    print("welfare_perturbed:", welfare_perturbed)
    print("obj_E_perturbed:", obj_E_perturbed)
    print("obj_R_perturbed:", obj_R_perturbed)
    print("obj_I_perturbed:", obj_I_perturbed)
    print()
    
    welfare_list.append(welfare)
    
    tau[:, :, i+1] = tau[:, :, i] - eta[i] * (d/delta[i]) * w_i[:-1].reshape((num_edges, T)) \
                        * (welfare_perturbed - welfare)
    
    B[i+1] = B[i] - eta[i] * (d/delta[i]) * w_i[-1] * (welfare_perturbed - welfare)
    
    print("tau[:, :, i+1], before projection:\n", tau[:, :, i+1])
    print("B[i+1], before projection:\n", B[i+1])
    
    tau[:, :, i+1], B[i+1] = proj_tau_B(T, num_edges, tau[:, :, i+1], B[i+1], od_to_edges_list_full, \
                                        tau_max = tau_upper_limit, B_max = B_upper_limit)
    
    print("tau[:, :, i+1], after projection:\n", tau[:, :, i+1])
    print("B[i+1], after projection:\n", B[i+1])
    
    if i >= diffs_num_cols + 2:
        tau_diffs = np.max(np.absolute(tau[:, :, i-diffs_num_cols : i-1] - tau[:, :, i-diffs_num_cols+1 : i]), axis = 2)
        B_diffs = B[i-diffs_num_cols : i-1] - B[i-diffs_num_cols+1 : i]
        
#         print("tau[:, :, 0:10]:", tau[:, :, 0:10])
#         print("B[0:10]:", B[0:10])
        print("tau_diffs:\n", tau_diffs)
        print("B_diffs:\n", B_diffs)
        print()
        
        if max(np.max(np.absolute(tau_diffs)), np.max(np.absolute(B_diffs))) < error_bound:
            print("Within error bound.")
            break

time_2 = time.time()

print()
print("Time:", time_2 - time_1)




Iter: 0
i:
 0
tau[:, :, i], before projection:
 [[0.09998507 0.1892127  0.28656039 0.09893891 0.04365383]
 [0.04052379 0.06717643 0.20833568 0.05774396 0.14996386]
 [0.79108695 0.66320932 0.98227094 0.06576718 0.78828095]
 [0.53572692 0.79683829 0.42663121 0.23536505 0.30208404]
 [0.74332596 0.46434049 1.20133794 1.00004619 0.38546419]
 [0.03842111 0.13033896 0.10387492 0.07271414 0.07829966]
 [0.15111417 0.1409052  0.11370494 0.09536731 0.13508253]]
B[i], before projection:
 3.6500274995601307
tau_perturbed[:, :, i], before projection:
 [[0.10003715 0.18887254 0.28631564 0.09895427 0.04386476]
 [0.04055506 0.06725484 0.20816493 0.05745411 0.14980591]
 [0.79113107 0.66286227 0.98272037 0.06584918 0.78884861]
 [0.53533612 0.79655015 0.42620605 0.23490713 0.30210015]
 [0.74272225 0.46492012 1.20136052 1.00017239 0.38570075]
 [0.03848824 0.13025399 0.10383768 0.07286954 0.07813242]
 [0.15085965 0.14087624 0.11378873 0.09510581 0.13513733]]
B_perturbed[i], before projection:
 3.6496433395


prob.status: optimal

prob.status: optimal

in_indices: [2, 3, 4]
el_indices: [0, 1]


in_indices: [2, 3, 4]
el_indices: [0, 1]

welfare: 787884.710992092
obj_E: 5918.742002480412
obj_R: 7337.62537952212
obj_I: 789303.5943691337

welfare_perturbed: 787879.2454178057
obj_E_perturbed: 5918.178819011888
obj_R_perturbed: 7336.875681577352
obj_I_perturbed: 789297.9422803712

tau[:, :, i+1], before projection:
 [[0.09761383 0.20608296 0.30046861 0.09179488 0.03240327]
 [0.04037586 0.06837387 0.2032647  0.06971765 0.15148583]
 [0.78946451 0.67911068 0.96449501 0.06503075 0.76598731]
 [0.54672648 0.80151352 0.45591562 0.24888165 0.30346833]
 [0.77058955 0.43627633 1.20854544 0.99350535 0.37551795]
 [0.04156313 0.14332608 0.10688908 0.05792986 0.08524689]
 [0.16596986 0.15161224 0.10460592 0.10564559 0.14283713]]
B[i+1], before projection:
 3.664124859610816

B: 3.664124859610816
B_max_wrt_tau_feas: 11.772235720926197
B_feas: 3.664124859610816

tau[:, :, i+1], after projection:
 [[0.0976 0.206


prob.status: optimal

prob.status: optimal

in_indices: [2, 3, 4]
el_indices: [0, 1]


in_indices: [2, 3, 4]
el_indices: [0, 1]

welfare: 787805.2206941623
obj_E: 5917.923889411557
obj_R: 7350.698697480946
obj_I: 789237.9955022316

welfare_perturbed: 787814.634838674
obj_E_perturbed: 5918.03197339842
obj_R_perturbed: 7350.078353444947
obj_I_perturbed: 789246.6812187205

tau[:, :, i+1], before projection:
 [[0.09824288 0.20758486 0.29780525 0.09095395 0.03518985]
 [0.0434133  0.06759833 0.20359142 0.06711636 0.15015722]
 [0.79212224 0.68358666 0.96737605 0.06772187 0.7666781 ]
 [0.54689853 0.80110786 0.45845912 0.24868532 0.30417277]
 [0.77408009 0.43477889 1.20893227 0.99205443 0.37226612]
 [0.04685263 0.14073363 0.1091005  0.05876515 0.08484027]
 [0.16657416 0.14952671 0.105866   0.10518997 0.14359785]]
B[i+1], before projection:
 3.661265859547105

B: 3.661265859547105
B_max_wrt_tau_feas: 11.791620595211945
B_feas: 3.661265859547105

tau[:, :, i+1], after projection:
 [[0.0982 0.207

In [ ]:
# dict_latency_params["Belmont"]

In [77]:
min_welfare = min(welfare_list)
argmin_welfare_list = welfare_list.index(min(welfare_list))
argmin_tau = tau[:, :, argmin_welfare_list]
argmin_B = B[argmin_welfare_list]

print("argmin_B:", argmin_B)
print("B[argmin_welfare_list]:", B[argmin_welfare_list])
print("min_welfare:", min_welfare)
print()
print(welfare_list)

B

argmin_B: 3.6500274995601307
B[argmin_welfare_list]: 3.6500274995601307
min_welfare: 787415.0890120076

[787415.0890120076, 788516.9912248425, 787981.7199924351, 787884.710992092, 787878.7782898434, 787849.5796866879, 787805.2206941623, 787746.2903846442]


array([3.6500275, 3.6659   , 3.6643   , 3.6647   , 3.6641   , 3.6623   ,
       3.6629   , 3.6613   , 3.6614   , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 

In [ ]:
# index_next_init = argmin_welfare_list
# B_next_init = B[index_next_init]
# tau_next_init = tau[:, :, index_next_init]

# print("index_next_init:", index_next_init)
# print("B_next_init:", B_next_init)
# print("tau_next_init:", tau_next_init)


In [ ]:
# lambda combinations to try to simulate:
# (1, 1, 1)
# (5, 5, 1)
# (10, 10, 1)
# (5, 10, 1)
# (5, 15, 1)
# (10, 15, 1)
# (10, 20, 1)
# (20, 20, 1)
# (20, 30, 1)
# (20, 40, 1)

In [62]:

# argmin_tau = np.array([[0.7091, 1.8252, 0.3238, 0.1215, 0.1079],
#                        [1.8739, 0.7429, 0.0, 0.9112, 1.8739],
#                        [0.0, 3.0448, 0.0, 0.0, 3.9494],
#                        [2.9504, 2.377, 3.4944, 0.0, 0.0],
#                        [7.2392, 8.4126, 0.0, 6.819, 0.6135],
#                        [0.2836, 0.8496, 0.963, 0.9527, 0.5689],
#                        [0.1346, 0.0, 0.0, 0.0712, 0.5024]]) / 100

# # argmin_B = 0.1664
# argmin_B = 0.0

# # VoT_array[:, :, 0]

In [78]:
argmin_y = solve_CBCP_direct(T, num_edges, num_gp_lanes, \
                                    argmin_tau, argmin_B, od_to_edges_array, \
                                    demand_array_temp, VoT_array, num_el, coeff_input)

print("argmin_tau:", argmin_tau)
print("argmin_B:", argmin_B)
print("min_welfare:", min_welfare)
print()
print("argmin_y:\n")

argmin_y



prob.status: optimal
argmin_tau: [[0.09998507 0.1892127  0.28656039 0.09893891 0.04365383]
 [0.04052379 0.06717643 0.20833568 0.05774396 0.14996386]
 [0.79108695 0.66320932 0.98227094 0.06576718 0.78828095]
 [0.53572692 0.79683829 0.42663121 0.23536505 0.30208404]
 [0.74332596 0.46434049 1.20133794 1.00004619 0.38546419]
 [0.03842111 0.13033896 0.10387492 0.07271414 0.07829966]
 [0.15111417 0.1409052  0.11370494 0.09536731 0.13508253]]
argmin_B: 3.6500274995601307
min_welfare: 787415.0890120076

argmin_y:



{(0, 0, 0, 0, 0): 47.77779144047728,
 (0, 0, 0, 1, 0): 4.687088930116387e-06,
 (0, 0, 0, 2, 0): 0.0002938690349278521,
 (0, 1, 0, 0, 0): 27.56397525188679,
 (0, 1, 0, 1, 0): 1.4921189527277575e-05,
 (0, 1, 0, 2, 0): 0.0002925172704811249,
 (0, 2, 0, 0, 0): 3.970725349628734e-05,
 (0, 2, 0, 1, 0): 115.15740827677052,
 (0, 3, 0, 0, 0): 0.00012028392303243338,
 (0, 3, 0, 1, 0): 134.75859544206253,
 (0, 4, 0, 0, 0): 0.0016449617475405018,
 (0, 4, 0, 1, 0): 287.2794353813762,
 (0, 0, 0, 0, 1): 47.77794087100148,
 (0, 0, 0, 1, 1): 2.474160954782793e-06,
 (0, 0, 0, 2, 1): 0.00014665143870067174,
 (0, 1, 0, 0, 1): 27.56412887341847,
 (0, 1, 0, 1, 1): 7.915013635875442e-06,
 (0, 1, 0, 2, 1): 0.00014590191470093567,
 (0, 2, 0, 0, 1): 3.0360060379931934e-05,
 (0, 2, 0, 1, 1): 115.15741762396364,
 (0, 3, 0, 0, 1): 5.588925615217422e-05,
 (0, 3, 0, 1, 1): 134.7586598367294,
 (0, 4, 0, 0, 1): 0.0011919760770566954,
 (0, 4, 0, 1, 1): 287.2798883670467,
 (0, 0, 0, 0, 2): 47.777983006266076,
 (0, 0, 0,

In [64]:
# demand_array_temp

# tau_upper_limit

# B_upper_limit

# y_perturbed_values = solve_CBCP_direct(T, num_edges, num_gp_lanes, \
#                                            tau_perturbed[:, :, i], B_perturbed[i], od_to_edges_array, \
#                                            demand_array_temp, VoT_array, num_el, coeff_input)

# T
# num_edges
# num_gp_lanes
# B_perturbed[i]
# coeff_input
# od_to_edges_array
# demand_array_temp
# VoT_array
# num_el
# coeff_input

# np.minimum(np.array([1, 2]), 0.5)

# tau_upper_limit
# argmin_welfare_list

# Compute Cost Metrics

In [79]:
num_groups = demand_array.shape[1]

el_indices = list(range(num_el))
in_indices = list(range(num_el, num_groups))

edge_to_od_dict = {}
for e in range(num_edges):
    edge_to_od_dict[e] = [k for k in list(range(int(od_to_edges_array.shape[0]) )) \
                           if od_to_edges_array[k, 0] <= e <= od_to_edges_array[k, 1]]

argmin_x = {}
for e in range(num_edges):
    for t in range(T):
        argmin_x[(e, 0, t)] = 0
        argmin_x[(e, 0, t)] += sum(argmin_y[(od, g, e, 0, t)] + argmin_y[(od, g, e, 1, t)] \
                                   for od in edge_to_od_dict[e] for g in el_indices)
        argmin_x[(e, 0, t)] += sum(argmin_y[(od, g, e, 0, t)] \
                                   for od in edge_to_od_dict[e] for g in in_indices)
        
        argmin_x[(e, 1, t)] = 0
        argmin_x[(e, 1, t)] += sum(argmin_y[(od, g, e, 2, t)] \
                                   for od in edge_to_od_dict[e] for g in el_indices)
        argmin_x[(e, 1, t)] += sum(argmin_y[(od, g, e, 1, t)] \
                                   for od in edge_to_od_dict[e] for g in in_indices)

argmin_y_in_el_total = {}      
for e in range(num_edges):
    for t in range(T):
        argmin_y_in_el_total[(e, 0, t, 'el')] = sum(argmin_y[(od, g, e, 0, t)] + argmin_y[(od, g, e, 1, t)] \
                                                    for od in edge_to_od_dict[e] for g in el_indices)
        
        argmin_y_in_el_total[(e, 1, t, 'el')] = sum(argmin_y[(od, g, e, 2, t)] \
                                                    for od in edge_to_od_dict[e] for g in el_indices)
        
        argmin_y_in_el_total[(e, 0, t, 'in')] = sum(argmin_y[(od, g, e, 0, t)] \
                                                    for od in edge_to_od_dict[e] for g in in_indices)
        
        argmin_y_in_el_total[(e, 1, t, 'in')] = sum(argmin_y[(od, g, e, 1, t)] \
                                                    for od in edge_to_od_dict[e] for g in in_indices)


# y[(od, g, e, 0, t)] + y[(od, g, e, 1, t)] for od in edge_to_od_dict[e] for g in el_indice

# argmin_x

argmin_y_in_el_total

{(0, 0, 0, 'el'): 87.47403359122147,
 (0, 1, 0, 'el'): 508.84271165223697,
 (0, 0, 0, 'in'): 858.2330208175033,
 (0, 1, 0, 'in'): 3393.5538648776273,
 (0, 0, 1, 'el'): 354.7200879938319,
 (0, 1, 1, 'el'): 241.59665724962656,
 (0, 0, 1, 'in'): 533.7324364703786,
 (0, 1, 1, 'in'): 3718.0544492247527,
 (0, 0, 2, 'el'): 87.41567704946927,
 (0, 1, 2, 'el'): 508.9010681939892,
 (0, 0, 2, 'in'): 594.2567055739552,
 (0, 1, 2, 'in'): 3657.530180121175,
 (0, 0, 3, 'el'): 107.21187426024849,
 (0, 1, 3, 'el'): 489.10487098321,
 (0, 0, 3, 'in'): 838.0636581601411,
 (0, 1, 3, 'in'): 3413.7232275349897,
 (0, 0, 4, 'el'): 227.39737235157514,
 (0, 1, 4, 'el'): 368.9193728918833,
 (0, 0, 4, 'in'): 749.3233175615339,
 (0, 1, 4, 'in'): 3502.4635681335976,
 (1, 0, 0, 'el'): 201.74427281890058,
 (1, 1, 0, 'el'): 457.2141236330191,
 (1, 0, 0, 'in'): 884.0492253153247,
 (1, 1, 0, 'in'): 3631.2232237369794,
 (1, 0, 1, 'el'): 530.6003484112607,
 (1, 1, 1, 'el'): 128.3580480406588,
 (1, 0, 1, 'in'): 533.74724554

In [80]:
# coeff_input

print("argmin_tau[5, 3]:", argmin_tau[5, 3])
print("argmin_x[(5, 0, 3)]:", argmin_x[(5, 0, 3)])
print("argmin_x[(5, 1, 3)]:", argmin_x[(5, 1, 3)])


argmin_tau[5, 3]: 0.07271413844452555
argmin_x[(5, 0, 3)]: 1060.2008256598858
argmin_x[(5, 1, 3)]: 5584.86989560715


In [81]:
travel_times = {}

## coeff_input: const, slope, x-coordinate of transition point
# coeff_input = np.array([19.4, 0.1256, 0.786*1650]).reshape((3, 1)) @ np.ones((1, num_edges))

for e in range(num_edges):
    for t in range(T):
        travel_times[(e, 0, t)] = coeff_input[0, e] + coeff_input[1, e] * max(argmin_x[(e, 0, t)] - coeff_input[2, e], 0.0)
        travel_times[(e, 1, t)] = coeff_input[0, e] + coeff_input[1, e] * max(argmin_x[(e, 1, t)] - coeff_input[2, e], 0.0)

edge_demand = {}
avg_travel_time = {}
percent_on_express = {}
obj_E = np.zeros(num_edges)
obj_I = np.zeros(num_edges)
obj_R = np.zeros(num_edges)
obj = np.zeros(num_edges)

for e in range(num_edges):
    avg_travel_time[e, 'el'] = 0.0
    avg_travel_time[e, 'in'] = 0.0
    avg_travel_time[e, 'ex'] = 0.0
    avg_travel_time[e, 'gp'] = 0.0
    
    percent_on_express[e, 'el'] = 0.0
    percent_on_express[e, 'in'] = 0.0
    percent_on_express[e, 'all'] = 0.0
    
    obj_E[e] = 0.0
    obj_I[e] = 0.0
    obj_R[e] = 0.0
    obj[e] = 0.0
    
    for t in range(T):
        edge_demand[e, t, 'el'] = sum(argmin_y_in_el_total[(e, k, t, 'el')] for k in range(2))
        edge_demand[e, t, 'in'] = sum(argmin_y_in_el_total[(e, k, t, 'in')] for k in range(2))
        
    percent_on_express[e, 'el'] += sum(argmin_y_in_el_total[(e, 0, t, 'el')] for t in range(T)) \
                                    / sum(edge_demand[e, t, 'el'] for t in range(T))
    percent_on_express[e, 'in'] += sum(argmin_y_in_el_total[(e, 0, t, 'in')] for t in range(T)) \
                                    / sum(edge_demand[e, t, 'in'] for t in range(T))
    percent_on_express[e, 'all'] += sum(argmin_y_in_el_total[(e, 0, t, 'el')] + argmin_y_in_el_total[(e, 0, t, 'in')] for t in range(T)) \
                                    / sum(edge_demand[e, t, 'el'] + edge_demand[e, t, 'in'] for t in range(T))
    
    avg_travel_time[e, 'el'] += sum(argmin_y_in_el_total[(e, k, t, 'el')] * travel_times[(e, k, t)] for k in range(2) for t in range(T)) \
                                    / sum(edge_demand[e, t, 'el'] for t in range(T))    
    avg_travel_time[e, 'in'] += sum(argmin_y_in_el_total[(e, k, t, 'in')] * travel_times[(e, k, t)] for k in range(2) for t in range(T)) \
                                    / sum(edge_demand[e, t, 'in'] for t in range(T))    
#     avg_travel_time[e, 'ex'] += sum( (argmin_y_in_el_total[(e, 0, t, 'el')] + argmin_y_in_el_total[(e, 0, t, 'in')]) * travel_times[(e, 0, t)] for t in range(T)) \
#                                     / sum( argmin_y_in_el_total[(e, 0, t, 'el')] + argmin_y_in_el_total[(e, 0, t, 'in')] for t in range(T))
#     avg_travel_time[e, 'gp'] += sum( (argmin_y_in_el_total[(e, 1, t, 'el')] + argmin_y_in_el_total[(e, 1, t, 'in')]) * travel_times[(e, 0, t)] for t in range(T)) \
#                                     / sum( argmin_y_in_el_total[(e, 1, t, 'el')] + argmin_y_in_el_total[(e, 1, t, 'in')] for t in range(T))
    avg_travel_time[e, 'ex'] += sum(travel_times[(e, 0, t)] for t in range(T)) / T 
    avg_travel_time[e, 'gp'] += sum(travel_times[(e, 1, t)] for t in range(T)) / T 
    
    obj_E[e] = sum( argmin_y[(od, g, e, 0, t)] * VoT_array[od, g, t] * travel_times[e, 0, t] \
                        for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) \
                    + sum( argmin_y[(od, g, e, 1, t)] * (VoT_array[od, g, t] * travel_times[e, 0, t] + argmin_tau[e, t]) \
                        for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) \
                    + sum( argmin_y[(od, g, e, 2, t)] * VoT_array[od, g, t] * travel_times[e, 1, t] \
                          for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) 

    obj_I[e] = sum( argmin_y[(od, g, e, 0, t)] * (VoT_array[od, g, t] * travel_times[e, 0, t] + argmin_tau[e, t]) \
                        for od in edge_to_od_dict[e] for g in in_indices for t in range(T) ) \
                    + sum( argmin_y[(od, g, e, 1, t)] * VoT_array[od, g, t] * travel_times[e, 1, t] \
                        for od in edge_to_od_dict[e] for g in in_indices for t in range(T) )

    obj_R[e] = sum( argmin_y[(od, g, e, 0, t)] * argmin_tau[e, t] \
                        for od in edge_to_od_dict[e] for g in in_indices for t in range(T) ) \
                    + sum( argmin_y[(od, g, e, 1, t)] * argmin_tau[e, t] \
                        for od in edge_to_od_dict[e] for g in el_indices for t in range(T) )

    obj[e] = lambda_E * obj_E[e] - lambda_R * obj_R[e] + lambda_I * obj_I[e]


# welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, argmin_tau, \
#                 demand_array, VoT_array, num_el, od_to_edges_array, y, \
#                 coeff_input)

# avg_travel_time
# percent_on_express
# obj_R



In [82]:
# e = 0
# od = 0
# g = 0
# t = 0

# # sum( argmin_y[(od, g, e, 0, t)] * VoT_array[od, g, t] * travel_times[e, 0, t] \
# #                         for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) 
# sum( argmin_y[(od, g, e, 1, t)] * (VoT_array[od, g, t] * travel_times[e, 0, t] + tau[e, t]) \
#     for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) 
# #                     + sum( argmin_y[(od, g, e, 2, t)] * VoT_array[od, g, t] * travel_times[e, 1, t] \
# #                           for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) ) 

# # argmin_y[(od, g, e, 1, t)]



# # sum(edge_demand[e, t, 'el'] for t in range(T) )

In [83]:
# travel_times
argmin_x

{(0, 0, 0): 945.7070544087247,
 (0, 1, 0): 3902.3965765298644,
 (0, 0, 1): 888.4525244642105,
 (0, 1, 1): 3959.651106474379,
 (0, 0, 2): 681.6723826234245,
 (0, 1, 2): 4166.431248315164,
 (0, 0, 3): 945.2755324203896,
 (0, 1, 3): 3902.8280985181996,
 (0, 0, 4): 976.720689913109,
 (0, 1, 4): 3871.382941025481,
 (1, 0, 0): 1085.7934981342253,
 (1, 1, 0): 4088.4373473699984,
 (1, 0, 1): 1064.3475939520317,
 (1, 1, 1): 4109.8832515521935,
 (1, 0, 2): 631.0429897831784,
 (1, 1, 2): 4543.187855721046,
 (1, 0, 3): 1072.3634463847538,
 (1, 1, 3): 4101.86739911947,
 (1, 0, 4): 1002.0241821792763,
 (1, 1, 4): 4172.206663324947,
 (2, 0, 0): 981.0458752051279,
 (2, 1, 0): 5812.672052780535,
 (2, 0, 1): 1008.8679983614353,
 (2, 1, 1): 5784.849929624226,
 (2, 0, 2): 913.0381969010339,
 (2, 1, 2): 5880.679731084629,
 (2, 0, 3): 1188.327834531337,
 (2, 1, 3): 5605.390093454327,
 (2, 0, 4): 981.8356214863501,
 (2, 1, 4): 5811.882306499314,
 (3, 0, 0): 948.92983431381,
 (3, 1, 0): 5098.708462852644,
 (3

In [84]:
# print("argmin_x[(3, 0, 0)]:", argmin_x[(3, 0, 0)])
# print("argmin_x[(3, 1, 0)]:", argmin_x[(3, 1, 0)])

In [85]:
opt_data_array = np.zeros((num_edges, 16))

# argmin_tau
opt_data_array[:, 0:5] = argmin_tau

# argmin_tau_avg
opt_data_array[:, 5] = np.mean(argmin_tau, axis=1)

# B (as array)
opt_data_array[:, 6] = argmin_B * np.eye(1, num_edges, 0)

# percent_on_express (overall)
# percent_on_express (eligible)
# percent_on_express (ineligible)
opt_data_array[:, 7] = np.array([percent_on_express[e, 'all'] for e in range(num_edges)]) * 100
opt_data_array[:, 8] = np.array([percent_on_express[e, 'el'] for e in range(num_edges)]) * 100
opt_data_array[:, 9] = np.array([percent_on_express[e, 'in'] for e in range(num_edges)]) * 100

# avg_travel_time (express lane)
# avg_travel_time (general purpose lane)
opt_data_array[:, 10] = np.array([avg_travel_time[e, 'ex'] for e in range(num_edges)])
opt_data_array[:, 11] = np.array([avg_travel_time[e, 'gp'] for e in range(num_edges)])

# obj_E = {}
# obj_I = {}
# obj_R = {}
# obj
opt_data_array[:, 12] = np.array([obj_E[e] for e in range(num_edges)]) 
opt_data_array[:, 13] = np.array([obj_I[e] for e in range(num_edges)]) 
opt_data_array[:, 14] = np.array([obj_R[e] for e in range(num_edges)]) 
opt_data_array[:, 15] = np.array([obj[e] for e in range(num_edges)]) 


opt_data_array = np.round(opt_data_array, decimals=2)

In [86]:
column_names = []
column_names += ["tau (t=" + str(t+1) + ")" for t in range(T) ]
column_names += ["tau (time-averaged)", \
                 "B", \
                 "% overall users using express lanes", \
                 "% eligible users using express lanes", \
                 "% ineligible users using express lanes", \
                 "Average travel time (express lanes)", \
                 "Average travel time (general purpose lanes)", \
                 "Total travel cost (eligible users)", \
                 "Total travel cost (ineligible users)", \
                 "Total toll revenue", \
                 "Total societal cost"]

row_names = ["e=" + str(k+1) for k in range(num_edges) ]

df_opt_save = pd.DataFrame(opt_data_array, index=row_names, columns=column_names)

df_opt_save



,tau (t=1),tau (t=2),tau (t=3),tau (t=4),tau (t=5),tau (time-averaged),B,% overall users using express lanes,% eligible users using express lanes,% ineligible users using express lanes,Average travel time (express lanes),Average travel time (general purpose lanes),Total travel cost (eligible users),Total travel cost (ineligible users),Total toll revenue,Total societal cost
e=1,0.10,0.19,0.29,0.10,0.04,0.14,3.65,18.31,28.99,16.81,1.37,3.75,526.67,77559.23,472.72,77613.18
e=2,0.04,0.07,0.21,0.06,0.15,0.10,0.00,18.77,34.39,16.49,2.24,4.08,699.87,87598.79,371.64,87927.01
e=3,0.79,0.66,0.98,0.07,0.79,0.66,0.00,14.93,29.22,12.91,5.10,12.55,2806.20,349857.96,2425.40,350238.76
e=4,0.54,0.80,0.43,0.24,0.30,0.46,0.00,16.79,40.66,13.45,1.22,8.68,1375.47,197769.83,1326.43,197818.86
e=5,0.74,0.46,1.20,1.00,0.39,0.76,0.00,15.70,60.93,9.56,6.00,17.70,3293.68,575988.96,2210.62,577072.02
e=6,0.04,0.13,0.10,0.07,0.08,0.08,0.00,15.66,61.34,9.73,1.56,2.96,527.75,84386.50,208.09,84706.16
e=7,0.15,0.14,0.11,0.10,0.14,0.13,0.00,14.57,62.19,8.40,2.44,4.07,830.13,126347.59,336.36,126841.36


In [87]:
# e = 0
# t = 4

e = 4
t = 3

print("travel_times[(e, 0, t)]:\n", travel_times[(e, 0, t)])
print()
print("travel_times[(e, 0, t)] + argmin_tau[(e, t)] / np.max(VoT_array[:, in_indices, t]):\n", \
      travel_times[(e, 0, t)] + argmin_tau[(e, t)] / np.max(VoT_array[:, in_indices, t]) )
print()
print("travel_times[(e, 1, t)]:\n", travel_times[(e, 1, t)])
print()
print("Total ineligible user express lane flow at (e, t):\n", argmin_y_in_el_total[(e, 0, t, "in")])

VoT_array[:, in_indices, t]
# demand_array[:, in_indices]

# print("")



travel_times[(e, 0, t)]:
 5.889055830909846

travel_times[(e, 0, t)] + argmin_tau[(e, t)] / np.max(VoT_array[:, in_indices, t]):
 6.370799643725908

travel_times[(e, 1, t)]:
 17.811716812208257

Total ineligible user express lane flow at (e, t):
 776.1176001875525


array([[0.24101617, 0.62571892, 1.54212652],
       [0.25832216, 0.65645628, 1.93431756],
       [0.29931887, 0.54077108, 1.73624799],
       [0.2918613 , 0.57186466, 2.05823236],
       [0.24015915, 0.50981052, 1.85018558],
       [0.23177172, 0.65497837, 1.49681546],
       [0.22233093, 0.59049575, 2.07588799],
       [0.24461295, 0.52263426, 1.25586438],
       [0.24998678, 0.640692  , 1.4127142 ],
       [0.22790326, 0.60165649, 1.19646462],
       [0.20954086, 0.52809178, 1.39792827],
       [0.20710365, 0.66196679, 1.31930555],
       [0.28418036, 0.51957674, 1.38211991],
       [0.21010827, 0.56756946, 1.61557736],
       [0.23974482, 0.52427954, 1.93664154],
       [0.21811123, 0.56480827, 1.82758669],
       [0.25274915, 0.59388912, 1.53897188],
       [0.22422175, 0.66374543, 1.44719232],
       [0.30034591, 0.51285663, 1.36479344],
       [0.30110989, 0.66594679, 1.67987221],
       [0.25262164, 0.59464191, 1.88799321],
       [0.23164578, 0.60885474, 1.66741408],
       [0.

In [88]:
# obj_R

# for e in range(num_edges):
#     for t in range(T):
#         print(sum(argmin_y[(od, g, e, 0, t)] * argmin_tau[e, t] \
#                         for od in edge_to_od_dict[e] for g in in_indices) \
#                     + sum(argmin_y[(od, g, e, 1, t)] * argmin_tau[e, t] \
#                         for od in edge_to_od_dict[e] for g in el_indices) )

# VoT_array[:, :, 0]

In [89]:
random_string = ""
for idx_rand_str in range(10):
    random_string += str(np.random.randint(1, 9))

directory_to_save = "../data/opt_tolls_subsidies_metrics/"
random_filename = "opt___" + random_string + '.csv'

df_opt_save.to_csv(directory_to_save + random_filename)

# Compute good initializations for alpha:

In [90]:
# # el_indices
# # in_indices

# e = 0
# # VoT_array[edge_to_od_dict[e], in_indices, :]

# VoT_array_trunc_in_temp = VoT_array[edge_to_od_dict[e], :]
# VoT_array_trunc_in = VoT_array_trunc_in_temp[:, in_indices, :]

# # print("VoT_array[:, in_indices, :]:\n", VoT_array[:, in_indices, :])
# # print()
# # print("VoT_array[edge_to_od_dict[e], :, :]:\n", VoT_array[edge_to_od_dict[e], :, :])

# # VoT_array_trunc_in

In [91]:
# demand_array.shape

In [121]:
arr_temp = np.stack([np.array([1, 2, 3, 4, 5]), np.array([3, 1, 4, 5, 2]), np.array([2, 5, 1, 4, 3])]).T
print(arr_temp)
# arr_temp[0:, :]
# print(np.sum(arr_temp))

# arr_temp[[2, 3], [1, 2]]
arr_temp[np.ix_([2,3], [1,2])]

[[1 3 2]
 [2 1 5]
 [3 4 1]
 [4 5 4]
 [5 2 3]]


array([[4, 1],
       [5, 4]])

In [139]:
alpha_init = np.zeros((num_edges, T))

VoT_in_boundary = np.zeros((num_edges, T))
VoT_el_boundary = np.zeros((num_edges, T))

for e in range(num_edges):
#     print("edge_to_od_dict[e]:", edge_to_od_dict[e])
    
    for t in range(T):
        
        ## Ineligible users:
        
        VoT_array_trunc_in = np.zeros((len(edge_to_od_dict[e]), len (in_indices)))
        demand_array_trunc_in = np.zeros((len(edge_to_od_dict[e]), len (in_indices)))
        
#         for od_index, od_temp in enumerate(edge_to_od_dict[e]):
#             for g_index, g_temp in enumerate(in_indices):
#                 VoT_array_trunc_in[od_index, g_index] = VoT_array[od_temp, g_temp, t] 
#                 demand_array_trunc_in[od_index, g_index] = demand_array[od_temp, g_temp] 
        
#         VoT_array_trunc_in_temp = VoT_array[edge_to_od_dict[e], :, t]
#         VoT_array_trunc_in = VoT_array_trunc_in_temp[:, in_indices]
#         demand_array_trunc_in_temp = demand_array[edge_to_od_dict[e], :]
#         demand_array_trunc_in = demand_array_trunc_in_temp[:, in_indices]
        
        VoT_array_trunc_in = VoT_array[:, :, t][np.ix_(edge_to_od_dict[e], in_indices)]
        demand_array_trunc_in = demand_array[np.ix_(edge_to_od_dict[e], in_indices)]

#         print("np.sum(demand_array_trunc_in):", np.sum(demand_array_trunc_in))
#         print("(Same sum computed directly):", sum([demand_array[od, g] for od in edge_to_od_dict[e] \
#                                                     for g in in_indices]))

        assert np.abs(np.sum(demand_array_trunc_in) - sum([demand_array[od, g] for od in edge_to_od_dict[e] \
                                                    for g in in_indices])) <= 1E-3
#         assert np.sum(demand_array_trunc_in_flattened) == sum([demand_array[od, g] for od in edge_to_od_dict[e] \
#                                                             for g in in_indices])
        
#         print("VoT_array_trunc_in.shape:", VoT_array_trunc_in.shape)
#         print("demand_array_trunc_in.shape:", demand_array_trunc_in.shape)
#         print("len([demand_array[od, g] for od in edge_to_od_dict[e] for g in in_indices])", \
#               len([demand_array[od, g] for od in edge_to_od_dict[e] for g in in_indices]))
#         print("demand_array_trunc_in:", demand_array_trunc_in)
#         print("[demand_array[od, g] for od in edge_to_od_dict[e] for g in in_indices]:", [demand_array[od, g] for od in edge_to_od_dict[e] for g in in_indices])

        VoT_array_trunc_in_flattened = VoT_array_trunc_in.flatten()
        demand_array_trunc_in_flattened = demand_array_trunc_in.flatten()
        VoT_demand_in = np.stack([VoT_array_trunc_in_flattened, demand_array_trunc_in_flattened]).T    
        VoT_demand_in_sorted = VoT_demand_in[np.argsort(VoT_demand_in[:, 0])]
        VoT_demand_in_cumul = np.zeros(VoT_demand_in_sorted.shape)
        VoT_demand_in_cumul[:, 0] = VoT_demand_in_sorted[:, 0]
        for row_index in range(VoT_demand_in_cumul.shape[0]):
            VoT_demand_in_cumul[row_index, 1] = np.sum(VoT_demand_in_sorted[row_index:, 1])
        
    
        
        assert np.abs(np.sum(VoT_demand_in[:, 1]) - sum(demand_array[od, g] for od in edge_to_od_dict[e] \
                                                            for g in in_indices)) <= 1E-3
        assert np.abs(np.sum(VoT_demand_in_sorted[:, 1]) - sum(demand_array[od, g] for od in edge_to_od_dict[e] \
                                                            for g in in_indices)) <= 1E-3
        assert np.abs(VoT_demand_in_cumul[0, 1] - sum(demand_array[od, g] for od in edge_to_od_dict[e] \
                                                for g in in_indices)) <= 1E-3

        ## Eligible users:

        # Flatten VoT_array_trunc_in and demand_array_trunc_in, and concatenate.
        # Sort by VoT
        # Create cumulative demand array

#         VoT_array_trunc_el_temp = VoT_array[edge_to_od_dict[e], :, t]
#         VoT_array_trunc_el = VoT_array_trunc_el_temp[:, el_indices]
#         demand_array_trunc_el_temp = demand_array[edge_to_od_dict[e], :]
#         demand_array_trunc_el = demand_array_trunc_el_temp[:, el_indices]

        VoT_array_trunc_el = VoT_array[:, :, t][np.ix_(edge_to_od_dict[e], el_indices)]
        demand_array_trunc_el = demand_array[np.ix_(edge_to_od_dict[e], el_indices)]

        VoT_array_trunc_el_flattened = VoT_array_trunc_el.flatten()
        demand_array_trunc_el_flattened = demand_array_trunc_el.flatten()
        VoT_demand_el = np.stack([VoT_array_trunc_el_flattened, demand_array_trunc_el_flattened]).T    
        VoT_demand_el_sorted = VoT_demand_el[np.argsort(VoT_demand_el[:, 0])]
        VoT_demand_el_cumul = np.zeros(VoT_demand_el_sorted.shape)
        VoT_demand_el_cumul[:, 0] = VoT_demand_el_sorted[:, 0]
        for row_index in range(VoT_demand_el_cumul.shape[0]):
            VoT_demand_el_cumul[row_index, 1] = np.sum(VoT_demand_el_sorted[row_index:, 1])
    
        print("VoT_demand_el_cumul[0, 1]:", VoT_demand_el_cumul[0, 1])
        print("(Same sum computed directly):", sum(demand_array[od, g] for od in edge_to_od_dict[e] \
                                                for g in el_indices))
        print()
        assert np.abs(VoT_demand_el_cumul[0, 1] - sum(demand_array[od, g] for od in edge_to_od_dict[e] \
                                                for g in el_indices)) <= 1E-3

    
#     VoT_array[od, g, t]


# for e in range(num_edges):
#     for t in range(T):
        
# for e in [0]:
#     for t in [0]:
        
        print()
        print("e:", e)
        print("t:", t)
        
        ## Ineligible users:
        
        # Find minimum entry in cumulative demand array such that demand >= argmin_y_in_el_total[(e, 0, t, 'in')]
        # Find corresponding VoT index -> Value
        
        VoT_in_boundary_index = max([row_index for row_index in range(VoT_demand_in_cumul.shape[0]) \
                                     if VoT_demand_in_cumul[row_index, 1] >= argmin_y_in_el_total[(e, 0, t, 'in')]])
        VoT_in_boundary[e, t] = VoT_demand_in_cumul[VoT_in_boundary_index, 0]
        
        ## Eligible users:
        
        # Find minimum entry in cumulative demand array such that demand >= argmin_y_in_el_total[(e, 0, t, 'in')]
        # Find corresponding VoT index -> Value
        
        print("Total el user flow at e, t:", \
                 sum(demand_array[(od, g)] for od in edge_to_od_dict[e] for g in el_indices))
        print("argmin_y_in_el_total[(e, 0, t, 'el')]]:", argmin_y_in_el_total[(e, 0, t, 'el')])
        print("VoT_demand_el_cumul[:, 1]", VoT_demand_el_cumul[:, 1])
        print("VoT_demand_el_cumul[0, 1]:", VoT_demand_el_cumul[0, 1])
        print("(Same sum computed directly):", sum(demand_array[od, g] for od in edge_to_od_dict[e] \
                                                for g in el_indices))
        print()
        assert np.abs(VoT_demand_el_cumul[0, 1] - sum(demand_array[od, g] for od in edge_to_od_dict[e] \
                        for g in el_indices)) <= 1E-3
        
        VoT_el_boundary_index = max([row_index for row_index in range(VoT_demand_el_cumul.shape[0]) \
                                     if VoT_demand_el_cumul[row_index, 1] >= argmin_y_in_el_total[(e, 0, t, 'el')]+ 1E-3])
        VoT_el_boundary[e, t] = VoT_demand_el_cumul[VoT_el_boundary_index, 0]
        
        if argmin_y_in_el_total[(e, 0, t, 'el')] <= 1E-3:
            alpha_init[e, t] = 0
        else:
            alpha_init[e, t] = 1.0 - VoT_el_boundary[e, t]/VoT_in_boundary[e, t]
        
        print("VoT_el_boundary_index:", VoT_el_boundary_index)
        print("VoT_in_boundary_index:", VoT_in_boundary_index)
        print("VoT_el_boundary[e, t]:", VoT_el_boundary[e, t])
        print("VoT_in_boundary[e, t]:", VoT_in_boundary[e, t])
        print("alpha_init[e, t]:", alpha_init[e, t])
        
        assert 0.0 <= alpha_init[e, t] <= 1.0


VoT_demand_el_cumul[0, 1]: 596.3167467964665
(Same sum computed directly): 596.3167467964665


e: 0
t: 0
Total el user flow at e, t: 596.3167467964665
argmin_y_in_el_total[(e, 0, t, 'el')]]: 87.47403359122147
VoT_demand_el_cumul[:, 1] [596.3167468  548.53865668 540.89193293 452.12730357 432.16894355
 356.50399037 337.58784116 218.16466346 190.6003807  186.18880931
 134.97844622 123.46400774  79.81115014  68.89798713]
VoT_demand_el_cumul[0, 1]: 596.3167467964665
(Same sum computed directly): 596.3167467964665

VoT_el_boundary_index: 11
VoT_in_boundary_index: 18
VoT_el_boundary[e, t]: 0.10238603988603988
VoT_in_boundary[e, t]: 2.042296664152623
alpha_init[e, t]: 0.9498672050524446
VoT_demand_el_cumul[0, 1]: 596.3167467964665
(Same sum computed directly): 596.3167467964665


e: 0
t: 1
Total el user flow at e, t: 596.3167467964665
argmin_y_in_el_total[(e, 0, t, 'el')]]: 354.7200879938319
VoT_demand_el_cumul[:, 1] [596.3167468  548.53865668 540.89193293 452.12730357 432.16894355
 356.503990

In [94]:
# # VoT_array_trunc_in_flattened
# # demand_in_trunc_in_flattened

# print("argmin_y_in_el_total[(e, 0, t, el)]:\n", argmin_y_in_el_total[(e, 0, t, 'el')])
# print()
# print("argmin_y_in_el_total[(e, 0, t, in)]:\n", argmin_y_in_el_total[(e, 0, t, 'in')])
# print()
# print("VoT_demand_el_cumul:\n", VoT_demand_el_cumul)
# print()
# print("VoT_demand_in_cumul:\n", VoT_demand_in_cumul)



In [140]:
# argmin_tau
# alpha_init

init_tau_alpha_array = np.block([argmin_tau, alpha_init])
init_tau_alpha_array

array([[0.09998507, 0.1892127 , 0.28656039, 0.09893891, 0.04365383,
        0.94986721, 0.98230519, 0.9508927 , 0.95025535, 0.98389598],
       [0.04052379, 0.06717643, 0.20833568, 0.05774396, 0.14996386,
        0.94986721, 0.98230519, 0.95086844, 0.94706865, 0.94843263],
       [0.79108695, 0.66320932, 0.98227094, 0.06576718, 0.78828095,
        0.94984244, 0.94622398, 0.9449713 , 0.94713217, 0.94891659],
       [0.53572692, 0.79683829, 0.42663121, 0.23536505, 0.30208404,
        0.94649313, 0.94622398, 0.98426437, 0.98348985, 0.94358402],
       [0.74332596, 0.46434049, 1.20133794, 1.00004619, 0.38546419,
        0.98329854, 0.98462258, 0.98426437, 0.94797562, 0.94358402],
       [0.03842111, 0.13033896, 0.10387492, 0.07271414, 0.07829966,
        0.98288625, 0.98474398, 0.9843886 , 0.94797562, 0.94654435],
       [0.15111417, 0.1409052 , 0.11370494, 0.09536731, 0.13508253,
        0.94776304, 0.98474398, 0.9843886 , 0.94864021, 0.94874389]])

In [142]:
column_names = []
column_names += ["tau (t=" + str(t+1) + ")" for t in range(T) ]
column_names += ["alpha (t=" + str(t+1) + ")" for t in range(T) ]

row_names = ["e=" + str(k+1) for k in range(num_edges) ]

df_inits_save = pd.DataFrame(init_tau_alpha_array, index=row_names, columns=column_names)

# df_inits_save

random_string = ""
for idx_rand_str in range(10):
    random_string += str(np.random.randint(1, 9))

directory_to_save = "../data/opt_tolls_subsidies_metrics/"
random_filename = "inits___" + random_string + '.csv'

df_inits_save.to_csv(directory_to_save + random_filename)

In [99]:
# argmin_y_in_el_total[(e, 0, t, 'in')]
# demand_array_trunc_in_flattened

for e in range(num_edges):
    for t in range(T):
        print("e:", e)
        print("t:", t)
        print("argmin_y_in_el_total[(e, 0, t, 'el')]:", argmin_y_in_el_total[(e, 0, t, 'el')])
        print()

e: 0
t: 0
argmin_y_in_el_total[(e, 0, t, 'el')]: 87.47403359122147

e: 0
t: 1
argmin_y_in_el_total[(e, 0, t, 'el')]: 354.7200879938319

e: 0
t: 2
argmin_y_in_el_total[(e, 0, t, 'el')]: 87.41567704946927

e: 0
t: 3
argmin_y_in_el_total[(e, 0, t, 'el')]: 107.21187426024849

e: 0
t: 4
argmin_y_in_el_total[(e, 0, t, 'el')]: 227.39737235157514

e: 1
t: 0
argmin_y_in_el_total[(e, 0, t, 'el')]: 201.74427281890058

e: 1
t: 1
argmin_y_in_el_total[(e, 0, t, 'el')]: 530.6003484112607

e: 1
t: 2
argmin_y_in_el_total[(e, 0, t, 'el')]: 20.14574612034743

e: 1
t: 3
argmin_y_in_el_total[(e, 0, t, 'el')]: 190.51685087952504

e: 1
t: 4
argmin_y_in_el_total[(e, 0, t, 'el')]: 190.02535676153585

e: 2
t: 0
argmin_y_in_el_total[(e, 0, t, 'el')]: 142.97119901894135

e: 2
t: 1
argmin_y_in_el_total[(e, 0, t, 'el')]: 355.85292877034067

e: 2
t: 2
argmin_y_in_el_total[(e, 0, t, 'el')]: 265.5710908208926

e: 2
t: 3
argmin_y_in_el_total[(e, 0, t, 'el')]: 292.22691067396994

e: 2
t: 4
argmin_y_in_el_total[(e, 0, t,

# Testing solve CBCP direct with a small example:

In [ ]:
# od_to_edges_array = np.array([[0, 0], [0, 1], [1, 1], [1, 2]])
# od_to_edges_list_full = [[0], [0, 1], [1], [1, 2]]
# edge_to_ods = [[0, 1], [1, 2, 3], [3]]

# num_od = 3
# num_edges = 3
# num_gp_lanes = 3
# num_groups = 5
# T = 5
# el_indices = [0, 1]
# in_indices = [2, 3, 4]
# B = 3
# tau = np.zeros((num_edges, T))

# for e in range(num_edges):
#     for t in range(T):
#         tau[e, t] = 1.5 + 0.5 * e + 0.2 * t

# demand_array = np.zeros((num_od, num_groups))
# VoT_array = np.zeros((num_od, num_groups, T))

# for od in range(num_od):
#     for g in range(num_groups):
#         demand_array[od, g] = 1.0 + od + 0.1 * g
#         VoT_array[od, g, t] = 2.0 + od + 0.1 * g
        
# coeff_input = np.array([1, 1, 0, 0, 0])
# latency_params_length = coeff_input.shape[0]

# ex_to_gp_multiplier = np.array([1/num_gp_lanes**p for p in range(latency_params_length)]).reshape((latency_params_length, 1)) \
#                         @ np.ones((1, num_edges))
# a = np.zeros((latency_params_length, num_edges, 2))
# coeff[:, :, 0] = coeff_input.reshape((latency_params_length, 1)) @ np.ones((1, num_edges))
# coeff[:, :, 1] = coeff[:, :, 0] * ex_to_gp_multiplier


# y_vals = solve_CBCP_direct(T, num_edges, num_gp_lanes, tau, B, od_to_edges_array, \
#                            demand_array, VoT_array, num_el, coeff_input)


# # y = {}
# # for od in range(num_od):
# #     for e in od_to_edges_list_full[od]:
# #         for t in range(T):
# #             for g in el_indices:
# #                 for k in [0, 1, 2]:
# #                     y[(od, g, e, k, t)] = cp.Variable(1)
# #             for g in in_indices:
# #                 for k in [0, 1]:
# #                     y[(od, g, e, k, t)] = cp.Variable(1)

# # x = {}
# # for e in range(num_edges):
# #     for k in [0, 1]:
# #         for t in range(T):
# #             x[(e, k, t)] = cp.Variable(1)

# # # Objective:
# # func = 0.0
# # for e in range(num_edges):

# # for od in range(num_od):
# #     for e in od_to_edges_list_full[od]:
# #         for t in range(T):
# #             for g in el_indices:
# #                 func += tau[e, t] * y[(od, g, e, 1, t)] / VoT_array[od, g, t]
# #             for g in in_indices:
# #                 func += tau[e, t] * y[(od, g, e, 0, t)] / VoT_array[od, g, t]

# # objective = cp.Minimize(func)

# # # Constraints:
# # constraints = []

# # constraints += [y[(od, g, e, k, t)] >= 0.0 for od in range(num_od) \
# #                 for e in od_to_edges_list_full[od] for g in el_indices  \
# #                 for k in [0, 1, 2] for t in range(T)]
# # constraints += [y[(od, g, e, k, t)] >= 0.0 for od in range(num_od) \
# #                 for e in od_to_edges_list_full[od] for g in in_indices  \
# #                 for k in [0, 1] for t in range(T)]


# # for e in range(num_edges):
# #     for t in range(T):
# # #         print("e:", e)
# # #         print("edge_to_ods[e]:", edge_to_ods[e])

# #         ## Edge contraints:
# #         constraints += [sum( y[(od, g, e, 0, t)] + y[(od, g, e, 1, t)] for od in edge_to_ods[e] for g in el_indices) \
# #                             + sum( y[(od, g, e, 0, t)] for od in edge_to_ods[e] for g in in_indices ) \
# #                             == x[(e, 0, t)] ]
# #         constraints += [sum( y[(od, g, e, 2, t)] for od in edge_to_ods[e] for g in el_indices) \
# #                             + sum( y[(od, g, e, 1, t)] for od in edge_to_ods[e] for g in in_indices ) \
# #                             == x[(e, 1, t)] ]

# #         ## Group flow constraints:
# #         constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1, 2]) == demand_array[od, g] \
# #                         for od in edge_to_ods[e] for g in el_indices]
# #         constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1]) == demand_array[od, g] \
# #                         for od in edge_to_ods[e] for g in in_indices]

# # constraints += [sum(y[(od, g, e, 0, t)] * tau[e, t] for e in od_to_edges_list_full[od] for t in range(T)) \
# #                 <= B * demand_array[od, g] for od in range(num_od) for g in el_indices]

# # # Problem:
# # prob = cp.Problem(objective, constraints)

# # # Solve:
# # result = prob.solve()


# # assert prob.status != "infeasible", "problem.status should not be infeasible."
# # assert prob.status != "unbounded", "problem.status should not be unbounded."
# # print()
# # print("prob.status:", prob.status)

# # # Extract Values:
# # y_values = {}
# # for e in range(num_edges):
# #     for od in edge_to_ods[e]:
# #         for t in range(T):
# #             for g in el_indices:
# #                 for k in [0, 1, 2]:
# #                     print("y[(od, g, e, k, t)].value:", y[(od, g, e, k, t)].value)
# #                     y_values[(od, g, e, k, t)] = max(y[(od, g, e, k, t)].value[0], 0.0)
# #             for g in in_indices:
# #                 for k in [0, 1]:
# #                     print("y[(od, g, e, k, t)].value:", y[(od, g, e, k, t)].value)
# #                     y_values[(od, g, e, k, t)] = max(y[(od, g, e, k, t)].value[0], 0.0)




In [ ]:
# demand_edges_array

In [ ]:
# np.sum(demand_array[3:16, :])/5

In [ ]:
print("first(welfare_list):", welfare_list[0])
print("min(welfare_list):", min(welfare_list))
print("max(welfare_list):", max(welfare_list))
print("argmin_tau:", argmin_tau)
print("argmin_B:", argmin_B)

## Test:

## <font color='red'>Colored Font Titles</font> 

# Scratch Work:

In [ ]:
x = cp.Variable(2)
y = cp.Variable(2)
v_fixed = np.array([0, 1])
objective = cp.Minimize(cp.sum_squares(x - y) + cp.sum_squares(x - v_fixed))
constraints = []
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print("x.value:", x.value)
print("y.value:", y.value)
print()


## CVXPY can handle 4d arrays:

In [ ]:

I, J, K, L = 2, 3, 4, 5

# Variables:
x_test = {}
for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                x_test[(i, j, k, ell)] = cp.Variable(1)
            
# Objective:
func = 0.0
func += cp.sum([x_test[(i, j, k, ell)]**2 for i in range(I) for j in range(J) \
                for k in range(K) for ell in range(L)])
            
objective = cp.Minimize(func)

# Constraints:
constraints = []

for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                constraints += [cp.sum([x_test[(i, j, k, ell)] for i in range(I) for j in range(J) \
                                        for k in range(K) for ell in range(L) ]) == 1.0]
                constraints += [x_test[(i, j, k, ell)] >= 0.0 for i in range(I) for j in range(J) \
                                        for k in range(K) for ell in range(L)]

# Solve problem:
prob = cp.Problem(objective, constraints)
result = prob.solve()

# Print solution:
for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                print("i, j, k, ell:", i, j, k, ell)
                print("x_test[(i,j,k, ell)].value:", x_test[(i, j, k, ell)].value)


## Linear Approximation for Latency Function:

In [ ]:
# Variables:
v = cp.Variable(1)
            
# Objective:
func = v - 1 + cp.square(cp.maximum(v-1, 0))
objective = cp.Minimize(func)

# Constraints:
constraints = [-3.0 <= v, v <= 3.0]

# Solve problem:
prob = cp.Problem(objective, constraints)
result = prob.solve()

# Print solution:
print("v.value:", v.value)
